Inducibility
=============

The followsing cells maximize the induced density of the claimed graphs as they appear in the table, in a few cases with the help of a consturction. The proved bound and the density of the target on the optimal construction (or one of them, if there are multiple) is printed, to verify that the lower and upper bounds match.

In [1]:
G = GraphTheory
G.printlevel(0)

In [2]:
#3-star plus isolated vertex
target = G(5,edges=[[0,1],[0,2],[0,3]])
R = QQ[sqrt(3)]; sq3 = R(sqrt(3))
r1=1/4+sq3/12
r2=1/4-sq3/12
constr = G.blowup_construction(7, [r1, r1, r2, r2], edges=[[0, 1], [2, 3]])
bound = G.optimize(target, 7, exact=True, construction=constr, file="certificates/ind0")
value = constr.density(target)
print("Value on construction is {}, flag algebra bound is {}".format(value, bound))

Value on construction is 5/24, flag algebra bound is 5/24


In [3]:
#4-cycle plus isolated vertex
target = G(5,edges=[[0,1],[1,2],[2,3],[3,0]])
R = QQ[sqrt(3)]; sq3 = R(sqrt(3))
r1=1/4+sq3/12
r2=1/4-sq3/12
constr = G.blowup_construction(7, [r1, r1, r2, r2], edges=[[0, 1], [2, 3]])
bound = G.optimize(target, 7, exact=True, file="certificates/ind1")
value = constr.density(target)
print("Value on construction is {}, flag algebra bound is {}".format(value, bound))

Value on construction is 5/32, flag algebra bound is 5/32


In [5]:
#4-cycle plus joint edge
target = G(5, edges=[[0, 1], [1, 2], [2, 3], [3, 0], [0, 4]])
constr = G.blowup_construction(5, 2, edges={(0, 1): 5/6})
bound = G.optimize(target, 5, exact=True, construction=constr, file="certificates/ind2")
value = constr.density(target)
print("Value on construction is {}, flag algebra bound is {}".format(value, bound))

Value on construction is 15625/62208, flag algebra bound is 15625/62208


Stability check
================

First cells includes a short script that checks condition 3 and condition i from Theorem ...

Then each certificate is checked for these conditions using a provided type $\tau$.

Finally, the certificate for the 4-cycle with a joined edge is inspected separately, to verify the
stability of the probabilistic construction.

In [6]:
from fractions import Fraction
from sage.algebras.combinatorial_theory import _unflatten_matrix
import pickle

def to_sage(dim, data):
    if dim==0:
        if isinstance(data, Fraction):
            return QQ(data)
        if isinstance(data, float):
            return RR(data)
        return data
    return [to_sage(dim-1, xx) for xx in data]

def check_stability(file, H, construction_size, construction_edges, ftype):
    print("\n\nchecking stability for " + file)
    G.reset()
    G.printlevel(0)
    if not file.endswith(".pickle"):
        file += ".pickle"
    with open(file, "rb") as f:
        certificate = pickle.load(f)
    target_size = certificate["target size"]
    original_bound = to_sage(0, certificate["result"])
    
    # Checking condition 2.a
    ftype_untyped = ftype.subflag(ftype_points=[])
    G.exclude(ftype_untyped)
    try:
        target = H.afae()
    except:
        target = 0
    if target==0: 
        bound = 0
    else:
        try:
            bound = G.optimize(target, target_size-1, exact=True, denom=2**20, construction=[])
        except:
            bound = 1
        if bound > original_bound:
            bound = G.optimize(target, target_size, exact=True, denom=2**20, construction=[])
    G.reset()
    
    if bound < original_bound:
        print(" - condition 2.a is satisfied")
    else:
        print(" - condition 2.a is not satisfied")
        return

    # Checking condition 3
    construction = G.blowup_construction(target_size, construction_size, edges=construction_edges)
    cvals = construction.values()
    svals = to_sage(1, certificate["slack vector"])
    correct_slacks = True
    for ii in range(len(svals)):
        if svals[ii]==0 and cvals[ii]==0:
            correct_slacks = False
    if not correct_slacks:
        print(" - condition 3 is not satisfied")
        return
    else:
        print(" - condition 3 is satisfied")

    # Checking condition i
    index = -1
    for ii,xx in enumerate(certificate["typed flags"]):
        if xx[1] == ftype._pythonize():
            index = ii
            break
    if index==-1:
        print(" - type not found")
        return
    mat = matrix(to_sage(2, _unflatten_matrix(certificate["X matrices"][index])[0]))
    if mat.nullity()==1:
        print(" - condition i is satisfied")
        print("problem is stable")
        return
    else:
        print(" - condition i is not satisfied, the matrix has nullity", mat.nullity())

    # Checking condition ii
    B_edges = [xx for xx in construction_edges if xx[0]!=xx[1]]
    B = G(construction_size, edges=B_edges)
    G.exclude(B)
    try:
        target = H.afae()
    except:
        target = 0

    if target==0: 
        bound = 0
    else:
        try:
            bound = G.optimize(target, target_size-1, exact=True, denom=2**20, construction=[])
        except:
            bound = 1
        if bound > original_bound:
            bound = G.optimize(target, target_size, exact=True, denom=2**20, construction=[])
    G.reset()
    if bound < original_bound:
        print(" - condition ii is satisfied")
        print("problem is stable")
    else:
        print(" - condition ii is not satisfied")

In [7]:
cherry_edge_type = G(5, edges=[[0, 2], [0, 3], [1, 4]], ftype=[0, 1, 2, 3, 4])

check_stability("certificates/ind0", 
                G(5,edges=[[0,1],[0,2],[0,3]]), 
                4, [[0, 1], [2, 3]], 
                cherry_edge_type)

check_stability("certificates/ind1", 
                G(5,edges=[[0,1],[1,2],[2,3],[3,0]]),
                4, [[0, 1], [2, 3]], 
                cherry_edge_type)



checking stability for certificates/ind0
 - condition 2.a is satisfied
 - condition 3 is satisfied
 - condition i is not satisfied, the matrix has nullity 4
 - condition ii is satisfied
problem is stable


checking stability for certificates/ind1
 - condition 2.a is satisfied
 - condition 3 is satisfied
 - condition i is not satisfied, the matrix has nullity 2
 - condition ii is satisfied
problem is stable


In [9]:
#
# The data for the "4-cycle plus joint edge" argument
#
with open("certificates/ind2.pickle", "rb") as f:
    cert = pickle.load(f)

ftypes = G._get_relevant_ftypes(5)

_, ft0, _ = ftypes[0]
X0 = matrix(to_sage(2, _unflatten_matrix(cert["X matrices"][0])[0]))
print("\nFor condition A: nullity(Xtau) = {}  where tau=({})".format(X0.nullity(), ft0))

_, ft3, _ = ftypes[3]
X3 = matrix(to_sage(2, _unflatten_matrix(cert["X matrices"][3])[0]))
print("\nFor condition B: nullity(Xtau) = {}  where tau=({})".format(X3.nullity(), ft3))

ts1, ft1, _ = ftypes[1]
X1 = matrix(to_sage(2, _unflatten_matrix(cert["X matrices"][1])[0]))
base1 = [xx for xx in G.generate(ts1, ft1) if len(xx.blocks("edges"))<=1]
print("\nFor condition C: list of tau flags with <=1 connection: ")
print("\n".join(map(str, base1)))
space1 = matrix([xx.afae().values() for xx in base1])
print("\nIntersection of this space with Xtau is \n{}".format(X1.kernel().intersection(space1.image())))


ts2, ft2, _ = ftypes[2]
X2 = matrix(to_sage(2, _unflatten_matrix(cert["X matrices"][2])[0]))
base2 = [xx for xx in G.generate(ts2, ft2) if len(xx.blocks("edges"))<=2]
print("\nFor condition D: list of tau flags with <=1 connection: ")
print("\n".join(map(str, base2)))
space2 = matrix([xx.afae().values() for xx in base2])
print("\nIntersection of this space with Xtau is \n{}".format(X2.kernel().intersection(space2.image())))


For condition A: nullity(Xtau) = 1  where tau=(Ftype on 1 points with edges=())

For condition B: nullity(Xtau) = 1  where tau=(Ftype on 3 points with edges=(01 02))

For condition C: list of tau flags with <=1 connection: 
Flag on 4 points, ftype from (0, 1, 2) with edges=()
Flag on 4 points, ftype from (0, 2, 3) with edges=(01)
Flag on 4 points, ftype from (2, 0, 3) with edges=(01)
Flag on 4 points, ftype from (2, 3, 0) with edges=(01)

Intersection of this space with Xtau is 
Vector space of degree 8 and dimension 0 over Rational Field
Basis matrix:
[]

For condition D: list of tau flags with <=1 connection: 
Flag on 4 points, ftype from (0, 1, 2) with edges=(01)
Flag on 4 points, ftype from (0, 1, 2) with edges=(01 03)
Flag on 4 points, ftype from (1, 0, 2) with edges=(01 03)
Flag on 4 points, ftype from (0, 2, 1) with edges=(02 13)

Intersection of this space with Xtau is 
Vector space of degree 8 and dimension 0 over Rational Field
Basis matrix:
[]


In [28]:
p = 5/6
gamma = p^5 * (1-p) / 2^4
var("x0 x1 y0 y1")
uf_emb_half = x0 * x1^2 * y0 * p^2 * (1-p)^2 + x0^3 * y1 * p^2 * (1-p) + \
2*(x0^2 * y1^2 * p^3 * (1-p)) + x0^2 * y0 * y1 * p^3 + x0 * y0 * y1^2 * p^4
uf_emb = uf_emb_half + uf_emb_half.subs(x0=x1, x1=x0, y0=y1, y1=y0)

q = uf_emb.subs(y0 = 1/2 - x0, y1 = 1/2 - x1)
print("q(0, 5/12) = {} and 5*gamma = {}".format(q.subs(x0 = 0, x1 = 5/12), 5*gamma))

print("\n\n\nTesting the interior")

R.<x, y, z> = PolynomialRing(QQ, 3, order='lex')
qab = q.subs(x0=x, x1=y)
gens = [qab.derivative(x), qab.derivative(y), z-qab]
print("\nIdeal generators:")
I = R.ideal(gens)
for p in gens:
    print("\n", p)

Gb = I.groebner_basis()

print("\nGröbner Basis:\n")
for p in Gb:
    print("\n", p)

print("\nSingle variable generator, Q(z) is", Gb[-1]/(factor(Gb[-1]).unit()))

print("\nIs 5*gamma a root? ", Gb[-1].subs(z=5*gamma)==0)


print("\n\n\nTesting y=1/2 edge")
print("\nOptimums are")
for xx in solve(q.subs(x0=1/2).derivative(x1)==0, x1):
    opt = xx.rhs()
    val = q.subs({x0:1/2, x1:opt})
    print("x={}, gives same as 5*gamma? {}".format(opt, bool(val==5*gamma)))

print("\n\n\nTesting y=0 edge")
print("\nOptimums are")
for xx in solve(q.subs(x0=0).derivative(x1)==0, x1):
    opt = xx.rhs()
    val = q.subs({x0:0, x1:opt})
    print("x={}, gives same as 5*gamma? {}".format(opt, bool(val==5*gamma)))

print("\n\n\nTesting corners")
print("\nValues are")
for xx, yy in [(0, 0), (1/2, 0), (1/2, 1/2)]:
    val = q.subs({x0:xx, x1:yy})
    print("x={}, y={} gives same as 5*gamma? {}".format(xx, yy, bool(val==5*gamma)))

q(0, 5/12) = 15625/746496 and 5*gamma = 15625/746496



Testing the interior

Ideal generators:

 125/432*(2*x - 1)*x*(2*y - 1) + 25/216*x^2*(2*y - 1) - 625/10368*(2*x - 1)*(2*y - 1)^2 - 125/5184*x*(2*y - 1)^2 - 625/2592*(2*x - 1)*(2*y - 1)*y - 25/1296*x*(2*y - 1)*y + 475/2592*(2*x - 1)*y^2 - 25/1296*x*y^2 + 125/432*(2*y - 1)*y^2 - 25/216*y^3

 125/432*(2*x - 1)*x^2 - 25/216*x^3 - 625/10368*(2*x - 1)^2*(2*y - 1) - 625/2592*(2*x - 1)*x*(2*y - 1) + 475/2592*x^2*(2*y - 1) - 125/5184*(2*x - 1)^2*y - 25/1296*(2*x - 1)*x*y - 25/1296*x^2*y + 125/432*(2*x - 1)*(2*y - 1)*y + 25/216*(2*x - 1)*y^2

 -125/864*(2*x - 1)*x^2*(2*y - 1) + 25/432*x^3*(2*y - 1) + 625/10368*(2*x - 1)*x*(2*y - 1)^2 - 125/2592*x^2*(2*y - 1)^2 + 625/10368*(2*x - 1)^2*(2*y - 1)*y + 25/2592*x^2*(2*y - 1)*y - 125/2592*(2*x - 1)^2*y^2 + 25/2592*(2*x - 1)*x*y^2 - 125/864*(2*x - 1)*(2*y - 1)*y^2 + 25/432*(2*x - 1)*y^3 + z

Gröbner Basis:


 x + y - 5187891984106164236712945182528392771109995899625841179383300096/27579621898765897

Verify the certificates
====================
The following cell verifies that the provided certificates are correct.
The cell can be run independently of the rest of the notebook.

In [1]:
G = GraphTheory
G.printlevel(0)
print("\nFor the 3-star plus isolated vertex")
G.verify("certificates/ind0")
print("\nFor the 4-cycle plus isolated vertex")
G.verify("certificates/ind1")
print("\nFor the 4-cycle plus joint edge")
G.verify("certificates/ind2")


For the 3-star plus isolated vertex
The solution is valid, it proves the bound 5/24

For the 4-cycle plus isolated vertex
The solution is valid, it proves the bound 5/32

For the 4-cycle plus joint edge
The solution is valid, it proves the bound 15625/62208


15625/62208